<h1 align="middle">CS 184: Computer Graphics and Imaging, Spring 2022</h1>
  <h1 align="middle">Project 3-1: Path Tracer</h1>
  <h2 align="middle">Rui Wang, Zhuowen Chen, CS184-CalVisitor</h2>

<br><br>

<h2 align="middle">Overview</h2>
    <p>An overview of the project, including your approach to and implementation for each of the parts, as well as what problems you have encountered and how you solved them. Strive for clarity and succinctness.</p>


<h2 align="middle">Part 1: Ray Generation and Scene Intersection</h2>

<b>First</b>, we walk through the ray generation and primitive intersection parts of the rendering pipeline:<br><br>
To begin with, we are given by normalized image coordinates (x, y) and we want to output a <b>Ray</b>
in the <b>world space</b>. By convention, we will first transform the <b>image coordinates</b> to
<b>camera space</b>, generate a ray in the <b>camera space</b>, and finally convert the ray into a ray 
in the <b>world space</b>. <br>
The sensor lies on the plane $Z = -1$. The center of the sensor is at $(0, 0, -1)$, the bottom left
corner is $(-tan(\frac{hFov}{2}), -tan(\frac{vFov}{2}), -1)$, the top right corner is $(tan(\frac{hFov}{2}), tan(\frac{vFov}{2}), -1)$<br>

So, $$x\_camera = (x-0.5) \times 2 \times tan(\frac{hFov}{2})$$ $$y\_camera = (y-0.5) \times 2 \times tan(\frac{vFov}{2})$$ 

We construct the <b>dire_camera</b>
vector as <b>{ x_camera, y_camera, -1 }</b>, then we use <b>c2w * dire_camera</b> to get the direction vector
<b>dire_world</b>, then we normalize the <b>dire_world</b>. Then, we generate the ray. The ray starts at
the camera position, which is in world space, and its direction is the <b>dire_world</b> vector that we
just got. Finally, we set the <b>min_t</b> and <b>max_t</b> attributes of the ray to be <b>nclip</b> and
<b>fclip</b> respectively.<br><br>

For primitive intersections, we calculate t and see if it lies within the interval $[r.min\_ t, r.max\_ t]$. If it is not, we take it as no intersection (invalid), otherwise we take it as a valid intersection, and set r.max_t to be the intersected t. If given an <b>Intersection</b> structure, we update its attributes.<br><br>

To get the ray intersection with <b>sphere</b>, we solve the equation 

$$(o+td-c)^2 - R^2 = 0 $$ 

Here, 
<b>o</b> is the origin of ray, <b>d</b> is the direction of ray, <b>c</b> is the center of the 
circle, <b>R</b> is the radius of the circle. So, we get 

$$t = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$$

Here, $a = d \cdot d, b = 2(o-c) \cdot d, c = (o-c) \cdot (o-c) - R^2$.
We see the value of $\sqrt{b^2 - 4ac}$ to see if there is intersection. If $\sqrt{b^2 - 4ac} < 0$, then there is not intersection. Otherwise, there are intersections.



To get the ray intersection with <b>triangle</b>, we take advantage of <b>Möller-Trumbore algorithm</b> (mentioned in discussion).

$$\begin{bmatrix} t \\ b_1 \\ b_2 \end{bmatrix} = \frac{1}{S_1 \cdot E_1} \begin{bmatrix} S_2 \cdot E_2 \\ S_1 \cdot S \\ S_2 \cdot D \end{bmatrix}$$
where $P_0, P_1, P_2$ are the three vertices of the triangle, $O$ is the origin of ray, $D$ is the direction of the ray, $E_1 = P_1 - P_0$, $E_2 = P_2 - P_0$, $S = O - P_0$, $S_1 = D \times E_2$, $S_2 = S \times E_1$.<br>
$b_1$ and $b_2$ are barycentric coordinates of the point corresponding to $P_1$ and $P_2$. We have to check if $t \ge 0, 0 \le b_1 \le 1, 0 \le b_2 \le 1, 0 \le (1-b_1-b_2) \le 1$ to see if there is intersection.
<br>

Also, note that in sphere and triangle intersections, we also need to check (as stated above) if the calculated t are in the valid range as a final check.

<b>Secondly</b>, we explain the triangle intersection algorithm we implemented in our own words: <br><br>
The algorithm is already represented in the above part. What the algorithm does is to first use barycentric coordinate to write a point $P$ within a triangle as 

$$P = (1-b_1-b_2)P_0 + b_1P_1 + b_2P_2 = P_0 + b_1(P_1 - P_0) + b_2(P_2 - P_0)$$

Then, we take the formula of a ray, to get the equation:

$$O + tD = P_0 + b_1(P_1 - P_0) + b_2(P_2 - P_0)$$
$$O - P_0 = -tD + b_1(P_1 - P_0) + b_2(P_2 - P_0)$$

Now, we get a matrix-vector equation:

$$Mx = b$$

where M = $\begin{bmatrix} -D & P_1 - P_0 & P_2 - P_0 \end{bmatrix}$, $x = \begin{bmatrix} t \\ b_1 \\ b_2 \end{bmatrix}$, $b = O - P_0$. We solve this matrix-vector equation to get the $t, b_1, b_2$, i.e. the t and its barycentric coordinates. We then check if the t and barycentric coordinates are in valid range to check if there is intersection.

<b>Thirdly</b>, we show images with normal shading for a few small <b>.dae</b> files:
<div align="middle">
    <table style="width=100%">
    <tr>
        <td>
        <img src="img/part1_CBspheres.png" align="middle" width="400px" />
        </td>
        <td>
        <img src="img/part1_cow.png" align="middle" width="400px" />
        </td>
    </tr>
    </table>
</div>

<h2 align="middle">Part 2: Bounding Volume Hierarchy</h2>

<b>First</b>, we walk through our BVH construction algorithm and xplain the heuristic we chose for picking the splitting point: <br>
<ul>
    <li>In the first part, we traverse through start to end, we expand the bounding box and count the number of primitives. Then, we check if the size is already smaller than or equal to <b>max_leaf_size</b>. If yes, we just set the start and end of the node to be the current start and end, and return. Otherwise, we go on.</li>
    <li>Next, we look for the axis (x,y,z) along which the bounding box is longgest. Then we sort the primitives by the magnitude of its centroid coordinate along the chosen axis.</li>
    <li>Finally, we use the first half of the list of primitives to construct the left child node of the current node, and the last half of the list of primitives to construct the right child node of the current node (both done by recursive call to this function). Then we return.</li>
</ul>
The heuristic is actually pretty simple. We just sort the primitives by its centroid coordinate along the axis along which the bounding box of all the given primitives is longest, then we use first and last half of the list of primitives to respetively construct left and right child node.

<b>Secondly</b>, we show images with normal shading for a few large <b>.dae</b> files that we can only render with BVH acceleration:
<div align="middle">
      <table style="width=100%">
        <tr>
          <td>
            <img src="img/part2_maxplanck.png" align="middle" width="400px" />
            <figcaption align="middle">maxplanck.dae</figcaption>
          </td>
          <td>
            <img src="img/part2_peter.png" align="middle" width="400px" />
            <figcaption align="middle">pater.dae</figcaption>
          </td>
        </tr>
        <br>
        <tr>
          <td>
            <img src="img/part2_beast.png" align="middle" width="400px" />
            <figcaption align="middle">beast.dae</figcaption>
          </td>
          <td>
            <img src="img/part2_CBlucy.png" align="middle" width="400px" />
            <figcaption align="middle">CBlucy.dae</figcaption>
          </td>
        </tr>
      </table>
    </div>

<b>Thirdly</b>, we compare rendering times on a few scenes with moderately complex geometries with and without BVH acceleration, and present our results in a one-paragraph analysis:
<div align="middle">
    <table style="width=100%;">
        <tr>
            <th>File Name</th>
            <th>Without BVH Acceleration</th>
            <th>With BVH Acceleration</th>
        </tr>
        <tr>
            <td>banana.dae</td>
            <td>22.6702s</td>
            <td>0.1594s</td>
        </tr>
        <tr>
            <td>cow.dae</td>
            <td>51.2285s</td>
            <td>0.2177s</td>
        </tr>
        <tr>
            <td>maxplanck.dae</td>
            <td>883.1857s</td>
            <td>0.3137s</td>
        </tr>
    </table>
</div>
When running the tests, I used parameters "-r 800 600" and my CPU is Intel Core i5-1135G7 @ 2.40GHz.<br>

After using BVH Tree structure and bounding box intersection test, we can see that there is a significantly large speedup. It has around 2800X speed up for maxplanck.dae, but not as much for smaller dae files like banana.dae and cow.dae, with only 100X to 300X speedup for them.

<h2 align="middle">Part 3: Direct Illumination</h2>

<b>First</b>, we walk through both implementations of the direct lighting function:<br>
<ul>
		<li>
			<b>estimate_direct_lighting_hemisphere</b>:
				<ol>
				<li>First, the starter code provided by the staff use the normal vector of the hit point in the <b>isect</b> to construct the coordinate system, and we get two matrices <b>o2w</b> and <b>w2o</b> so that we can transform vectors between object and world coordinates. The starter code also gives the <b>hit_p</b>, <b>w_out</b>, <b>num_samples</b> . </li>
				<li>
					Secondly, we get <b>num_samples</b> samples. In each iteration, we first get <b>w_in 
					= hemisphereSampler->get_sample()</b>, and we get <b>wj = o2w * w_in</b>. Here, w_in is 
					a random incoming light direction that we sampled, and wj is w_in converted to world 
					coordinates. Then, we initialize a new ray taking <b>hit_p</b> as origin, and <b>wj</b> 
					as direction. We also set its <b>min_t</b> as <b>EPS_F</b> to  alleviate numerical 
					precision issues that cause the ray to intersect the surface it started from. Then, we check
					is this ray intersects with some objects in the scene. If no, we just return. Otherwise, we
					calculate the <b>fr = isect.bsdf->f(w_out, w_in)</b>, <b>Li = temp_isect->bsdf->get_emission()</b>,
					<b>costheta = dot(isect.n, v)</b> where v is the normalized <b>wj</b>, and finally we have 
					<b>L_out += fr * Li * costheta / pdf</b> where <b>pdf = 1 / (2 * PI)</b>.
				</li>
				<li>
					Finally, we return <b>L_out / num_samples</b>.
				</li>
				</ol>
		</li>
		<li>
			<b>estimate_direct_lighting_importance</b>:
		</li>
	</ul>

<b>Secondly</b>, we show some images rendered with both implementations of the direct lighting function:
<div align="middle">
      <table style="width=100%">
        <tr>
          <td>
            <img src="img/part3_CBbunny_uni_hemi.png" align="middle" width="400px" />
            <figcaption align="middle">CBbunny.dae, Hemisphere</figcaption>
          </td>
          <td>
            <img src="img/part3_bunny_importance.png" align="middle" width="400px" />
            <figcaption align="middle">CBbunny.dae, Importance</figcaption>
          </td>
        </tr>
        <br>
        <tr>
          <td>
            <img src="img/part3_CBdragon_uni_hemi.png" align="middle" width="400px" />
            <figcaption align="middle">CBdragon.dae, Hemisphere</figcaption>
          </td>
          <td>
            <img src="img/part3_dragon_importance.png" align="middle" width="400px" />
            <figcaption align="middle">CBdragon.dae, Importance</figcaption>
          </td>
        </tr>
      </table>
    </div>

<b>Thirdly</b>, we focus on one particular scene with at least one area light and compare the noise levels in soft shadows when rendering with 1, 4, 16, and 64 light rays (the -l flag) and with 1 sample per pixel (the -s flag) using light sampling, not uniform hemisphere sampling. We used <b>CBbunny.dae</b>:
<div align="middle">
      <table style="width=100%">
        <tr>
          <td>
            <img src="img/part3_bunny_1_1.png" align="middle" width="400px" />
            <figcaption align="middle">CBbunny.dae, 1 light rays</figcaption>
          </td>
          <td>
            <img src="img/part3_bunny_1_4.png" align="middle" width="400px" />
            <figcaption align="middle">CBbunny.dae, 4 light rays</figcaption>
          </td>
        </tr>
        <br>
        <tr>
          <td>
            <img src="img/part3_bunny_1_16.png" align="middle" width="400px" />
            <figcaption align="middle">CBbunny.dae, 16 light rays</figcaption>
          </td>
          <td>
            <img src="img/part3_bunny_1_64.png" align="middle" width="400px" />
            <figcaption align="middle">CBbunny.dae, 64 light rays</figcaption>
          </td>
        </tr>
      </table>
    </div>

<b>Finally</b>, we compare the results between uniform hemisphere sampling and lighting sampling in a one-paragraph analysis:<br>
By comparing the graphs of the same dae file rendered with same resolution and same light rays per pixel sample (i.e. same parameters given when rendering), we can see that the uniform hemisphere sampling gives much more noise than the importance lighting sampling method gives overall, but it seems that the shadow of object rendered with importance sampling is more shattered.